In [1]:
# I'm not going to write a web application... because that's dumb.  But I'll do something similar.

from hashlib import sha1
from random import choices
from time import sleep, time

SECRET_KEY = b'\xe7\xef\x1e\x7f\xd7\x87\xa4\xeb\x10<\xd9\x9f\x8b\xec\x03\x8f'
DELAY = 0.01 #I'm doing 100ms instead of 50 just for convenience

def Validate(data: bytes, potential_hmac: bytes):
    hmac = sha1(SECRET_KEY+data).digest()
    assert len(potential_hmac) == len(hmac)
    for n in range(len(hmac)):
        sleep(DELAY)
        if hmac[n] != potential_hmac[n]:
            return False
    
    return True

def HMAC(data: bytes):
    return list(sha1(SECRET_KEY+data).digest())

In [2]:
# Make a random "file"

data = bytes(choices(range(256), k = 500))
print(data)

b'\x88$)\xb5\x92\xbe\x87to\x83\x1ayK\x86\x13\x16O\xb5O\xab"\xafR\x9d\xf8V\x0e\xcc\xbe\xab\xdb\x19\x1f>"!\xa5\xa6\xf4-I\x0c{\xbf\xbbv\x03\x8c\xf3\xf1Y\x99\xa4>\xda\xb1\xe6\xf4\x10<\xb8\x9b\xc66\x02\xe3\x82\t\xb5\xae\x1b\xec\xda\x8bn\xe1\xe3ay\x08\xf3\x7f\xc6\xd0\xc1\xf6\xb7\xbcZ@\xe0\xd0\xfe\r\xe5\x99\x13\xfa\xa5\x06\x8e`N\xbb\xd3\xc8\x15\xa2MR\xb7\x1f ts\xa9}\xfdt\x00\xc7\xa1\xce;z\x13\xd0\xc3\x97\x9e\xf7\x8e9\x16,&\xa9\x85\xbc\x8b+r\xa1\xbf\x96\xcf]\x83\x02\xc4 1\xfc\xae\x98\xa0|B\x93\x93\'$2r\xe8\x82\x18\x88k_\x12\x9b\xe8&~\xb3\r\xdb\xdbv\x9e\rP\xb9u\x82e$\x9ck\x0f~\xefp\x94\x9f\x87M\xc7\xb3\xf5;\x8a\xd8\xe7\x84aj\xb7~\xbb\xf2\x1c\xbf\x88k\xb9\x91\xfb\xea\x10=8jvMp\xc3\xa5\x03t"\xde\xb32\x9b\x1f\x94\xeb\x95\xf8\x11.\x1a\xa8\xca\xa8"\x14\xc7* \x95\xa6\xc6\x9d\xba\xd5;\xfe\xe4\t\xad\xd8\xff\x8a\x8a6\xa6\x07\xae\xf2\x0e.\xcd\xce\x98SF\x19\xbfyv\x94\x04\xef\x1b\xef\xed8\x0f!U&\x8b\x8a\xed\xc4\xbfr\xb4(,\x10(\x861a\xae\xa5\xf3^u\x18\xfd\xacz\x9bj\x1e\xaf\x0e\rTF\xe6H\x89\xb8\x08\xc8\xd7$H

In [3]:
print(HMAC(data))

[124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148, 15, 113, 217, 18, 184, 14, 196, 38]


In [4]:
known_bytes = []
TOTAL_HMAC_BYTES = 20
response = False

# The way I'm going to handle this is test each character 10 times and choose the one that had the total longest response.
# Taking multiple samples should help account for any random fluctuations in the timing, which I encountered previously.

for n in range(TOTAL_HMAC_BYTES):
    if response == True:
        break
    print('Known bytes:', known_bytes)
    dt = [0] * 256
    for char in [*range(256)]*10:
        guess = [ *known_bytes, char ] + [0] * ( TOTAL_HMAC_BYTES - len(known_bytes) - 1 )
        tic = time()
        response = Validate(data, guess)
        toc = time()
        if response == True:
            # I guess we're done
            known_bytes += [char]
            break
        else:
            dt[char] += toc-tic
    
    if response == False:
        max_dt = max(dt)
        best_char = dt.index(max_dt)
        known_bytes += [best_char]

            
    

Known bytes: []
Known bytes: [124]
Known bytes: [124, 84]
Known bytes: [124, 84, 246]
Known bytes: [124, 84, 246, 45]
Known bytes: [124, 84, 246, 45, 140]
Known bytes: [124, 84, 246, 45, 140, 39]
Known bytes: [124, 84, 246, 45, 140, 39, 89]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148, 15]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148, 15, 113]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148, 15, 113, 217]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148, 15, 113, 217, 18]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148, 15, 113, 217, 18, 184]
Known bytes: [124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148, 1

In [5]:
print(known_bytes)

[124, 84, 246, 45, 140, 39, 89, 49, 51, 17, 153, 148, 15, 113, 217, 18, 184, 14, 196, 38]


In [6]:
Validate(data, known_bytes)

True